In [1]:
import pandas as pd
import numpy as np
import regex as rg

In [2]:
folder = 'C:\\Users\\jessi\\Documents\\TCC Docs\\Datasets Covid\\'

In [3]:
#Dados Einstein
einstein_pacientes = pd.read_csv(folder + "EINSTEINAgosto\\EINSTEIN_Pacientes_2.csv", sep = "|", encoding="utf-8")
einstein_exames = pd.read_csv(folder + "EINSTEINAgosto\\EINSTEIN_Exames_2.csv", sep = "|", encoding="utf-8")

In [4]:
einstein_exames['DT_COLETA'].value_counts()

10/06/2020    22443
07/07/2020    21429
05/06/2020    21405
15/06/2020    21341
14/07/2020    21329
              ...  
03/11/2019     3559
23/02/2020     3507
31/12/2019     3105
25/12/2019     2998
01/01/2020     2619
Name: DT_COLETA, Length: 285, dtype: int64

In [5]:
df = pd.DataFrame()
df['EXAME'] = einstein_exames['DE_EXAME'] + ' - ' + einstein_exames['DE_ANALITO']
df['UNIDADE'] = einstein_exames['CD_UNIDADE']
df['VALOR_REF'] = einstein_exames['DE_VALOR_REFERENCIA']
technical_einstein = df.drop_duplicates()
technical_einstein.to_csv('Einstein_Exames.csv', index=False, sep="|",encoding='utf-8-sig')

In [6]:
einstein_exames['DT_COLETA'] = pd.to_datetime(einstein_exames['DT_COLETA'], format='%d/%m/%Y')
einstein_exames['DE_RESULTADO'].replace('','*',inplace=True)
to_drop = ['*', 'NOVA COLETA', 'Nova coleta', 'nova coleta', 'Nova Coleta', 'Indeterminado', 'Inconclusivo']
einstein_exames = einstein_exames[~einstein_exames['DE_RESULTADO'].isin(to_drop)]
#einstein_exames['DE_RESULTADO'].replace('*','',inplace=True)
#einstein_exames.loc[einstein_exames['DE_RESULTADO'].str.contains('oleta'), 'DE_RESULTADO'] = ''
#einstein_exames.replace(r'^\s*$', np.nan, regex=True, inplace = True)
einstein_exames['DE_EXAME'] = einstein_exames['DE_EXAME'] + " - " + einstein_exames['DE_ANALITO']
del einstein_exames['DE_ANALITO']
einstein_exames.sort_values(['ID_PACIENTE','DT_COLETA', 'DE_EXAME'], ascending=[False, False, False],inplace=True)

In [ ]:
einstein_exames.sort_by(values='DT_COLETA',ascending)

In [7]:
einstein_exames_exp = einstein_exames.pivot_table(index= ['ID_PACIENTE','DT_COLETA'], columns='DE_EXAME', values='DE_RESULTADO', aggfunc=';'.join)

In [8]:
cols_einstein = pd.DataFrame(einstein_exames_exp.notna().sum(),columns=['Preenchidos'])
cols_einstein['%'] = (cols_einstein['Preenchidos'] / einstein_exames_exp.shape[0]) * 100
del_cols = cols_einstein[cols_einstein['Preenchidos'] < 18000].sort_values(by='%', ascending=False).index.to_list()
del_cols.remove("Sorologia SARS-CoV-2/COVID19 IgG/IgM - COVID IgM Interp")
einstein_exames_exp.drop(del_cols, axis=1,inplace=True)

In [9]:
#einstein_exames_exp['COLUNAS_PREENCHIDAS'] = einstein_exames_exp.apply(lambda x: x.count(), axis=1)

In [10]:
cols_urina = [col for col in  einstein_exames_exp.columns if 'Urina' in col]
cols_hemograma = [col for col in  einstein_exames_exp.columns if 'Hemograma' in col]
df_einstein_exames = einstein_exames_exp[einstein_exames_exp[cols_urina[0]].notna() == True]
df_einstein_exames =  df_einstein_exames[df_einstein_exames[cols_hemograma[0]].notna() == True]

In [11]:
df_einstein_exames['PCR em tempo real para detecção de Coron - Resultado COVID-19:'].value_counts()

Não detectado                  1444
Detectado                       256
Não detectado;Não detectado       1
Name: PCR em tempo real para detecção de Coron - Resultado COVID-19:, dtype: int64

In [12]:
df_einstein_exames['DIAGNOSTICO_COVID'] = 'PCR - ' + df_einstein_exames['PCR em tempo real para detecção de Coron - Resultado COVID-19:']
df_einstein_exames.loc[df_einstein_exames['DIAGNOSTICO_COVID'].isna() & \
                       df_einstein_exames['Sorologia SARS-CoV-2/COVID19 IgG/IgM - COVID IgM Interp'].notna(),
                      'DIAGNOSTICO_COVID'] = 'Sorologia IgM - ' + df_einstein_exames['Sorologia SARS-CoV-2/COVID19 IgG/IgM - COVID IgM Interp']

In [13]:
df_einstein_exames['DIAGNOSTICO_COVID'].value_counts()

PCR - Não detectado                  1444
PCR - Detectado                       256
Sorologia IgM - Não reagente          107
Sorologia IgM - Reagente               21
Sorologia IgM -                         2
Sorologia IgM - Não Reagente            1
PCR - Não detectado;Não detectado       1
Name: DIAGNOSTICO_COVID, dtype: int64

In [14]:
df_einstein_exames = df_einstein_exames[~df_einstein_exames['DIAGNOSTICO_COVID'].isna()]
values_del = ['Sorologia IgM -  ', 'Sorologia IgM - Indeterminado']
df_einstein_exames = df_einstein_exames[~df_einstein_exames['DIAGNOSTICO_COVID'].isin(values_del)]

In [15]:
df_einstein_exames.to_csv('Einstein_v2.csv',sep="|",encoding='utf-8-sig')

In [16]:
df_einstein_exames

,DE_EXAME,Dosagem de Bilirrubina Total e Frações - Bilirrubina Direta,Dosagem de Bilirrubina Total e Frações - Bilirrubina Indireta,Dosagem de Bilirrubina Total e Frações - Bilirrubina Total,Dosagem de CreatiNINA - Creatinina,Dosagem de Cálcio Iônico - Cálcio Iônico mmol/L,Dosagem de Fosfatase Alcalina - Fosfatase Alcalina,Dosagem de Gama GT - Gama GT,Dosagem de Glicose - Dosagem de Glicose,Dosagem de Magnésio - Magnésio,Dosagem de Potássio - Potássio,...,Urina Tipo I - Hemoglobina Urina,Urina Tipo I - Hemácias Urina,Urina Tipo I - Leucócitos Urina,Urina Tipo I - Leveduras,Urina Tipo I - Nitrito,Urina Tipo I - Pigmentos biliares,Urina Tipo I - Proteína,Urina Tipo I - Urobilinogênio,Urina Tipo I - pH,DIAGNOSTICO_COVID
ID_PACIENTE,DT_COLETA,,,,,,,,,,,,,,,,,,,,,
0038fa1282332f290716cf10f631aad9bd594c74,2020-07-18,0.1,0.2,0.3,NaN,NaN,61,18,NaN,NaN,NaN,...,Ausente,4000,8000,Ausentes,Negativo,Ausentes,Não Detectado,Normal,5.0,PCR - Não detectado
005844cefaea9a1fde8a9816b37b53920ed6fd00,2020-06-28,0.1,0.2,0.3,1.11,1.18,NaN,NaN,NaN,NaN,4.1,...,Ausente,2000,<1000,Ausentes,Negativo,Ausentes,Não Detectado,Normal,7.0,PCR - Não detectado
005e37ee20c6d09138910aa1d980ff4644daa02c,2020-05-26,0.1,0.2,0.3,0.64,NaN,NaN,NaN,NaN,NaN,NaN,...,Ausente,2000,2000,Ausentes,Negativo,Ausentes,Não Detectado,Normal,5.0,PCR - Não detectado
00667cb4b8cd10735146cd02f4b987f93348ac86,2020-06-19,NaN,NaN,NaN,1.03,1.27,44,28,;90,1.6,NaN,...,Ausente,1000,<1000,Ausentes,Negativo,Ausentes,Não Detectado,Normal,5.0,PCR - Não detectado
006bac6d1c1a882deac98414d0c860f2b1ccd671,2020-05-02,NaN,NaN,NaN,1.05,NaN,NaN,NaN,NaN,NaN,NaN,...,Ausente,19000,7000,Ausentes,Negativo,Ausentes,Não Detectado,Normal,6,PCR - Não detectado
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ff847e6ac6fc30282772b31ac8d6cfb1fb8ede99,2020-07-13,0.1,0.3,0.4,0.75,1.06,123,95,;143,1.8,4.4,...,Ausente,<1000,<1000,Ausentes,Negativo,Ausentes,Não Detectado,Normal,7.0,PCR - Não detectado
ff8bce18f058659b525dd3d9cc3798854fea1aae,2020-06-16,NaN,NaN,NaN,0.91,1.24,NaN,NaN,76;,NaN,4.7,...,Ausente,<1000,3000,Ausentes,Negativo,Ausentes,Não Detectado,Normal,5.0,Sorologia IgM - Não reagente
ffc785516609746f65da2ff59bd2a07a947403a2,2020-06-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Presente,138000,3000,Ausentes,Negativo,Ausentes,Não Detectado,Normal,5.0,PCR - Não detectado


In [17]:
df_einstein_exames.shape

(1830, 56)

In [18]:
11816-9961

1855

In [19]:
len(df_einstein_exames.columns)

56